<a href="https://colab.research.google.com/github/janwaelty/Master-Thesis/blob/main/image_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import pandas as pd
import zipfile
import numpy as np
from google.colab import drive
import shutil
import torchvision



In [10]:
def load_image_data(zip_path, image_path):

  with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    file_list = zip_ref.namelist()
    file_list = [f for f in file_list if not f.startswith('__MACOSX')]
    zip_ref.extractall(extract_path, members=file_list)
  print(f"Dateien entpackt in: {extract_path}")

In [11]:
# Function to check if an image file exists
def check_image_existence(image_paths, base_dir):
    missing_images = []  # To store paths of missing images
    nans = 0
    invalid_url = 0
    for image_path in image_paths:
        # Skip if the path is not a valid string (e.g., NaN or None)
        if isinstance(image_path, str) and image_path.strip():  # Check if it's a valid non-empty string
            # Construct the full image path based on the base directory
            full_image_path = os.path.join(base_dir, image_path.lstrip('/'))  # Remove leading slash, if any

            # Check if the file exists
            if not os.path.exists(full_image_path):
                missing_images.append(full_image_path)  # Add to missing list if not found
                invalid_url += 1
        else:
            nans += 1
    print(f"number of nans: {nans}")
    print(f"missing images: {invalid_url}")
    return missing_images

def filter_data_for_missing_images(data, missing_images, base_dir):
    # Extract only relative paths from missing_images
    data = data.dropna(subset=['image_url'])
    missing_relative_paths = [os.path.relpath(img, base_dir) for img in missing_images]
    data_filtered = data[~data['image_url'].str.lstrip('/').isin(missing_relative_paths)]

    return data_filtered

In [12]:
def resnet_transform():
  return  torchvision.transforms.Compose([
    torchvision.transforms.Resize(256),            # Resize the image to be at least 256px
    torchvision.transforms.CenterCrop(224),        # Crop the center 224x224
    torchvision.transforms.ToTensor(),             # Convert the image to a tensor
    torchvision.transforms.Normalize(              # Normalize with ImageNet stats
        mean=[0.485, 0.456, 0.406],    # ImageNet mean values
        std=[0.229, 0.224, 0.225]      # ImageNet std values
    )])

In [ ]:
if __name__ == "__main__":


  # Mount Google Drive
  drive.mount('/content/drive')

  # Define source and destination paths
  source_path = "/content/drive/My Drive/Colab Notebooks/image_preprocess.ipynb"
  destination_path = "/content/drive/My Drive/image_preprocess.ipynb"

  # Check if the source file exists before moving
  if os.path.exists(source_path):
    # Move the file
    shutil.move(source_path, destination_path)
    print(f"Notebook moved to: {destination_path}")

  artist_data_path = "/content/drive/My Drive/artist_graph_data.xlsx"
  artwork_numeric_data = pd.read_excel(artist_data_path)

  zip_path = "/content/drive/My Drive/pic.zip"
  extract_path = "/content/data"
  load_image_data(zip_path, extract_path)
  # Check for missing images
  base_dir = "/content/data"
  image_paths = artwork_numeric_data['image_url'].values
  missing_images = check_image_existence(image_paths, base_dir)
  data_filtered = filter_data_for_missing_images(artwork_numeric_data, missing_images, base_dir)
  # check functioning of image data after filtering
  print(f"After filtering:{check_image_existence(data_filtered['image_url'],base_dir)} ")





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
